<a href="https://colab.research.google.com/github/nick-allen21/synthetic_patient_analysis/blob/main/nallen21_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS145: Project 1 | Project Name

## Author
* *Nicholas Allen, nallen21*

# Section 1: Project Overview

---
This project will anaylze the best practice in terms of cost and treatment efficacy for patients around the country as they choose a blend of providers and payors for there specific state. In order to complete this anlaysis, I am using synthetic patient data using synthea, a synthethic patient data creation git repo (cannot use / access real patient data to due PHI restrictions). Through this analysis, I hope to provide query frameworks creating actionable incomes for patients in how they pick providers, and payors given thier distinct state and income. The three questions I have chose are:
1.  ***Identify the insurer the best insurers for each state by using average encounter duration and price paid for each state.***
2.   ***Compare the average number of encounters per patient and the average base encounter cost per patient across income classes (Low, Mid, High).***
3. ***Measure the 30-day revisit rate across insurer–state combinations.***
---

More specifically, Question 1 surfaces which insurer–state pairings are most time-efficient and cost-effective, helping patients trade off shorter encounters against lower prices (or flagging outliers where long visits don’t translate to value). Question 2 normalizes utilization and spend by income tier, revealing whether certain groups face higher encounter burdens or costs per patient—evidence to inform targeted subsidies, plan design, or outreach. Question 3 highlights where care transitions may be breaking down by comparing 30-day revisit rates across insurers and states, pointing to concrete opportunities for better discharge planning, follow-up access, and network management. Together, these queries provide an actionable framework patients can use to choose providers and payers that maximize value given state context and income constraints.

Note that this analysis is largely centered around payors - in the healthcare architecture, many studies try to analyze a given hospital conglomerate and see how they are doing. This analysis actually takes a step back, and look at how the payment terms of a hosptial entity really influence the care they are able to provide. It is an acknowledgement of the corporate nature of our healthcare system that can provide insights on how to truly improve it.

# Section 2: Dataset Analysis

###2.1) Init Big Query Resources by connecting to bucket and creating tables

In [22]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery, storage

PROJECT_ID = "cs145-project-1-475101"
BUCKET = "nallen21_bucket_cs145"
DATASET_ID = "cs145_data"

bq_client = bigquery.Client(project=PROJECT_ID)
storage_client = storage.Client(project=PROJECT_ID)

# Create dataset if not already created
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = "US"
bq_client.create_dataset(dataset_ref, exists_ok=True)

Dataset(DatasetReference('cs145-project-1-475101', 'cs145_data'))

### 2.2) Detailed Overview

Using google cloud so cannot get table information from the schema, need to resort to gsutil to get the table schema information. Creating external tables at the same time


In [25]:
import subprocess
from google.cloud import bigquery

table_ids = [
    "conditions",
    "encounters",
    "medications",
    "observations",
    "patients",
    "payer_transitions",
    "payers",
    "providers",
]

for table_name in table_ids:
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}_ext"
    gcs_path = f"gs://{BUCKET}/{table_name}.csv"

    # Create external tables
    external_config = bigquery.ExternalConfig("CSV")
    external_config.source_uris = [gcs_path]
    external_config.autodetect = True
    external_config.options.skip_leading_rows = 1

    table = bigquery.Table(table_id)
    table.external_data_configuration = external_config

    bq_client.create_table(table, exists_ok=True)
    print(f"\nExternal table created: {table_id}")

    # get row count using dynamic loop query
    row_query = f"SELECT COUNT(*) AS row_count FROM `{table_id}`"
    row_count = bq_client.query(row_query).to_dataframe().iloc[0, 0]

    # Get GCS size with units
    # Use -s (bytes) so we can control the units later
    raw = subprocess.run(["gsutil", "du", "-s", gcs_path], capture_output=True, text=True).stdout.strip()

    if raw:
        bytes_size = int(raw.split()[0])
        mb = bytes_size / (1024 * 1024)
        # choose MB or GB units in print statement
        gcs_size = f"{mb:.2f} MB" if mb < 1024 else f"{mb/1024:.2f} GB"
    else:
        gcs_size = "N/A"

    print(f"{table_name}_ext — Rows: {row_count:,}, Size: {gcs_size}")



External table created: cs145-project-1-475101.cs145_data.conditions_ext
conditions_ext — Rows: 2,522,756, Size: 379.36 MB

External table created: cs145-project-1-475101.cs145_data.encounters_ext
encounters_ext — Rows: 4,850,448, Size: 1.51 GB

External table created: cs145-project-1-475101.cs145_data.medications_ext
medications_ext — Rows: 5,992,699, Size: 1.47 GB

External table created: cs145-project-1-475101.cs145_data.observations_ext
observations_ext — Rows: 48,847,506, Size: 8.17 GB

External table created: cs145-project-1-475101.cs145_data.patients_ext
patients_ext — Rows: 61,208, Size: 17.28 MB

External table created: cs145-project-1-475101.cs145_data.payer_transitions_ext
payer_transitions_ext — Rows: 2,438,672, Size: 392.72 MB

External table created: cs145-project-1-475101.cs145_data.payers_ext
payers_ext — Rows: 440, Size: 0.07 MB

External table created: cs145-project-1-475101.cs145_data.providers_ext
providers_ext — Rows: 40,002, Size: 7.23 MB


The main table is the patients table with all the unique identifiers for the patients. However, each patient has multiple interactions with the health system. As such, it is one of the smaller tables as it has one to many linkages with other tables in our analysis.

* **patients_ext** — One row per patient (primary key Id). Includes demographics like STATE and INCOME. All other clinical tables link back here via PATIENT.

* **encounters_ext** — One row per interaction/visit (primary key Id). Keys: PATIENT, PROVIDER, PAYER. Timestamps START/STOP, context (ENCOUNTERCLASS, CODE/ DESCRIPTION), and economics (BASE_ENCOUNTER_COST, TOTAL_CLAIM_COST, PAYER_COVERAGE). This is the hub for most utilization/cost analytics.

* **conditions_ext** — Problem list/history per patient and encounter (FKs: PATIENT, ENCOUNTER when applicable). Tracks diagnosis CODE/DESCRIPTION and onset/resolve dates; many-to-one with patients.

* **medications_ext** — Orders/admins tied to an encounter (FKs: PATIENT, ENCOUNTER). Contains drug DESCRIPTION, optional REASONDESCRIPTION/REASONCODE, and per-line costs (e.g., BASE_COST). Many meds per encounter.

* **observations_ext** — Measurements and results (FKs: PATIENT, ENCOUNTER). Includes CATEGORY (e.g., vital_signs, labs), CODE/DESCRIPTION, numeric VALUE/UNITS, and DATE. High-volume, many observations per encounter.

* **payer_transitions_ext** — Coverage history per patient (FK PATIENT). Captures when payer enrollment START/STOP changes; useful for attributing encounters to payers over time.

* **payers_ext** — Dimension table for payers/insurers (primary key Id, e.g., NAME). Join from encounters/payer_transitions on PAYER to get readable labels.

* **providers_ext** — Dimension table for providers/organizations (primary key Id). Join from encounters on PROVIDER to attribute utilization and costs to clinicians or sites.

*In short: patients is your grain for population; encounters is your grain for utilization and costs; conditions/medications/observations add clinical context to each encounter; payers/payer_transitions define who paid and when; providers tell you where/who delivered care.*


SQL: Show table sizes, row counts

## 2.3) Table Relationships

#### patients_ext
*   **primary_key:** `Id`

#### encounters_ext
*   **primary_key:** `Id`
*   **foreign key:** `PATIENT` to `Id` in patients_ext,

#### condition_ext
*   **foreign key:** `ENCOUNTER` to `Id` in encounters_ext
*   **foreign key:** `PATIENT` to `Id` in patients_ext,

#### payers_ext
*   **primary_key:** `Id`

#### payer_transitions_ext
*   **foreign key:** `PAYER` to `Id` in payers_ext
*   **foreign key:** `PATIENT` to `PATIENT` in patients_ext,

#### observation_ext
*   **foreign key:** `PATIENT` to `Id` in patients_ext,
*   **foreign key:** `ENCOUNTER` to `Id` in encounters_ext

#### medications_ext
*   **foreign key:** `PATIENT` to `PATIENT` in patients_ext
*   **foreign key:** `ENCOUNTER` to `Id` in encounters_ext









## Data Issues


SQL: Check for NULLs, duplicates

In [24]:
query = f"""
SELECT Id as null_patient_id
FROM {PROJECT_ID}.{DATASET_ID}.patients_ext
WHERE Id IS NULL
"""
df = bq_client.query(query).to_dataframe()
display(df)

query = f"""
SELECT COUNT(Id) as duplicate_id_patient_count
FROM {PROJECT_ID}.{DATASET_ID}.patients_ext
GROUP BY Id
HAVING COUNT(Id) > 1
"""
df = bq_client.query(query).to_dataframe()
display(df)

query = f"""
SELECT Id as null_encounters_id
FROM {PROJECT_ID}.{DATASET_ID}.encounters_ext
WHERE Id IS NULL
"""
df = bq_client.query(query).to_dataframe()
display(df)

query = f"""
SELECT COUNT(Id) as duplicate_encounters_id
FROM {PROJECT_ID}.{DATASET_ID}.encounters_ext
GROUP BY Id
HAVING COUNT(Id) > 1
"""
df = bq_client.query(query).to_dataframe()
display(df)

query = f"""
SELECT ENCOUNTER as null_condition_encounter, PATIENT as null_condition_patient
FROM {PROJECT_ID}.{DATASET_ID}.conditions_ext
WHERE (
  ENCOUNTER IS NULL
  Or
  PATIENT IS NULL
)
"""
df = bq_client.query(query).to_dataframe()
display(df)

query = f"""
SELECT PATIENT as null_observation_patient
FROM {PROJECT_ID}.{DATASET_ID}.observations_ext
WHERE (
  PATIENT IS NULL
)
"""
df = bq_client.query(query).to_dataframe()
display(df)

query = f"""
SELECT ENCOUNTER as null_observation_encounter
FROM {PROJECT_ID}.{DATASET_ID}.observations_ext
WHERE (
  ENCOUNTER IS NULL
)
"""
df = bq_client.query(query).to_dataframe()
display(df)


query = f"""
SELECT PATIENT as dup_payer_transitions_patient
FROM {PROJECT_ID}.{DATASET_ID}.payer_transitions_ext
GROUP BY PATIENT
HAVING COUNT(PATIENT) > 1
"""
df = bq_client.query(query).to_dataframe()
display(df)

,null_patient_id


,duplicate_id_patient_count


,null_encounters_id


,duplicate_encounters_id


,null_condition_encounter,null_condition_patient


,null_observation_patient


,null_observation_encounter
0,None
1,None
2,None
3,None
4,None
...,...
891982,None
891983,None
891984,None
891985,None


,dup_payer_transitions_patient
0,b5462c9a-0893-fad5-4a10-c99d6fe567cc
1,bf8cba8b-d398-a9b8-5a52-9666caa2ac9b
2,cd06b749-3b9e-22b7-49e6-bafdeff7747b
3,0c654c40-c1d7-bb91-c256-cf794a866fbc
4,9874751e-a4c0-4755-00fd-12b4c57db3ab
...,...
59992,9bd44063-a30b-6e76-0228-a67f32996fb1
59993,36b89681-41d8-5d04-77fb-acdf3710d237
59994,f08f8921-03b8-38e4-c988-47b6006d81e8
59995,6a9301d2-8534-9e22-b866-42a6a277995a


# Section 3: Get Your Feet Wet

*DELETE WHEN DONE READING: please write a tiny title and description for each query. Don't forget to add comments!*

*DELETE WHEN DONE READING: feel free to add more queries! But keep the format the same :)*

*DELETE WHEN DONE READING: In this cell, please specify which two queries you are uploading a debug table. Then underneath those queries, show the debug table. Below is an example:*

Debug Tabls for the following queries:
* Subquery 1: [your title]
* CTE 1: [your title]

## Subqueries

2 queries with scoped variables

### SubQuery 1:


Select all patients who have paid an average over twice the base encounter average cost

Please copy and paste your debug table image here for this query. Otherwise delete this cell.

In [4]:
query = f"""
SELECT DISTINCT p.FIRST, p.MIDDLE, p.LAST
FROM `{PROJECT_ID}.{DATASET_ID}.patients_ext` p
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.encounters_ext` e on p.Id = e.PATIENT
WHERE e.BASE_ENCOUNTER_COST > (
  SELECT AVG(e2.BASE_ENCOUNTER_COST) * 2 AS double_avg_cost
  FROM `{PROJECT_ID}.{DATASET_ID}.encounters_ext` AS e2
  WHERE e2.PATIENT = p.Id -- correlated to outer query, uses the patient id from the outer query
)
"""
df = bq_client.query(query).to_dataframe()
display(df)

,FIRST,MIDDLE,LAST
0,Alejandro916,Manuel446,Acevedo301
1,Lynda214,Lura184,Kreiger457
2,Hester117,Valda518,Legros616
3,Tom274,None,Wilkinson796
4,Zane918,Noel608,Rohan584
...,...,...,...
343,Lucas404,Marcus77,Erdman779
344,Otto672,None,Lindgren255
345,Mickey576,Owen89,Bradtke547
346,Quincy153,Willie882,Mertz280


### SubQuery 2:


Select all providers who insure personal plan owners

In [5]:
query = f"""
SELECT p.NAME
FROM `{PROJECT_ID}.{DATASET_ID}.payers_ext` p
WHERE EXISTS (
  SELECT 1
  FROM `{PROJECT_ID}.{DATASET_ID}.payer_transitions_ext` pt
  WHERE pt.PAYER = p.Id
    AND pt.PLAN_OWNERSHIP = 'Self'
)
"""
df = bq_client.query(query).to_dataframe()
display(df)

,NAME
0,Medicare
1,Medicaid
2,Dual Eligible
3,Humana
4,Blue Cross Blue Shield
...,...
391,Blue Cross Blue Shield
392,UnitedHealthcare
393,Aetna
394,Cigna Health


Please copy and paste your debug table image here for this query. Otherwise delete this cell.

In [6]:
# write code here
%%bigquery --project $project_id
# make sure you have the header above for each SQL cell

# here's an example of a SQL query using the NCAA basketball dataset in bigquery-public-data
SELECT id, market, name, mascot, mascot_name
FROM `bigquery-public-data.ncaa_basketball.mascots`
LIMIT 5;


ERROR:
 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/$project_id/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: e6e72984-760e-41dd-9d71-7674624192f2



## CTEs

2 queries with WITH clauses

### CTE 1:


Select all male new yorkers who are on medicaid

Please copy and paste your debug table image here for this query. Otherwise delete this cell.

In [7]:
# write code here
%%bigquery --project $project_id
# make sure you have the header above for each SQL cell

# here's an example of a SQL query using the NCAA basketball dataset in bigquery-public-data
SELECT id, market, name, mascot, mascot_name
FROM `bigquery-public-data.ncaa_basketball.mascots`
LIMIT 5;


ERROR:
 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/$project_id/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: a9e63d4f-6501-40ce-b943-a098ffbc9701



In [8]:
query = f"""
WITH male_new_yorkers AS (
  SELECT p.Id as patient_id, p.FIRST, p.MIDDLE, p.LAST, p.STATE, p.GENDER
  FROM `{PROJECT_ID}.{DATASET_ID}.patients_ext` p
  WHERE (
    p.GENDER = 'M'
      AND
    p.STATE = 'New York'
  )
),
male_medicaid_ny AS (
  SELECT mny.patient_id, mny.FIRST, mny.MIDDLE, mny.LAST,  mny.STATE, mny.GENDER
  FROM male_new_yorkers mny
  LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.payer_transitions_ext` pt on mny.patient_id = pt.PATIENT
  LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.payers_ext` p on pt.PAYER = p.Id
  WHERE p.NAME = 'Medicaid'
)
SELECT DISTINCT *
FROM male_medicaid_ny
"""
df = bq_client.query(query).to_dataframe()
display(df)

,patient_id,FIRST,MIDDLE,LAST,STATE,GENDER
0,30e02880-42cd-e6d8-3a17-c78a9fe8fc5f,Irvin970,Enoch803,Goyette777,New York,M
1,c552c29b-21d6-88a7-da33-6c391eba4a46,Ellis535,Danilo179,Olson653,New York,M
2,d1b92425-45da-0cc9-0c34-3989347111ea,Mose244,Ezra452,King743,New York,M
3,5d8b7af2-1c66-ff6c-cff9-dbdac595aada,Virgil85,Ryan260,Stoltenberg489,New York,M
4,dee82d76-77d9-112d-6bdc-f7183e52dc1b,Arturo47,None,Schmitt836,New York,M
...,...,...,...,...,...,...
321,96854dfe-367f-7911-a84b-806fe5bd02d9,Jordon466,Von197,Bode78,New York,M
322,b1862ba4-3354-4bb8-2f56-7dcde12cb611,Napoleon578,Reggie481,Turner526,New York,M
323,01ce5297-4143-1e88-a520-3c793a1b9ee3,Emory494,None,Will178,New York,M
324,4f641b5a-7852-065a-e052-5c4504ea9ea0,Courtney281,Sol312,Lindgren255,New York,M


### CTE 2:


Select vital sign observations that are above 150 dollar interactions

In [9]:
query = f"""
WITH vital_sign_encounters AS (
  SELECT o.ENCOUNTER, o.PATIENT, o.CATEGORY, o.DESCRIPTION
  FROM `{PROJECT_ID}.{DATASET_ID}.observations_ext` o
  WHERE
    o.CATEGORY = 'vital-signs'
),
expensive_vital_sign_encounters AS (
  SELECT vse.ENCOUNTER, vse.PATIENT, vse.CATEGORY, vse.DESCRIPTION, ee.BASE_ENCOUNTER_COST
  FROM vital_sign_encounters vse
  INNER JOIN `{PROJECT_ID}.{DATASET_ID}.encounters_ext` ee ON vse.ENCOUNTER = ee.Id
  WHERE ee.BASE_ENCOUNTER_COST > 150
)
SELECT DISTINCT vse.DESCRIPTION, vse.BASE_ENCOUNTER_COST
FROM expensive_vital_sign_encounters vse
"""
df = bq_client.query(query).to_dataframe()
display(df)

,DESCRIPTION,BASE_ENCOUNTER_COST
0,Pain severity - 0-10 verbal numeric rating [Sc...,152.15
1,Body mass index (BMI) [Ratio],152.15
2,Head Occipital-frontal circumference Percentile,152.15
3,Body Weight,166.97
4,Body mass index (BMI) [Percentile] Per age and...,166.97
...,...,...
127,Body Height,160.75
128,Body Weight,160.75
129,Body mass index (BMI) [Ratio],160.75
130,Head Occipital-frontal circumference,160.75


Please copy and paste your debug table image here for this query. Otherwise delete this cell.

In [10]:
# write code here
%%bigquery --project $project_id
# make sure you have the header above for each SQL cell

# here's an example of a SQL query using the NCAA basketball dataset in bigquery-public-data
SELECT id, market, name, mascot, mascot_name
FROM `bigquery-public-data.ncaa_basketball.mascots`
LIMIT 5;


ERROR:
 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/$project_id/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: d10fc7e6-a168-4050-a6b4-4b862ef63812



## Window Functions

3 queries with OVER, including RANK vs ROW_NUMBER. Please note the FAQs!

### Window Function 1:


Please copy and paste your debug table image here for this query. Otherwise delete this cell.

In [11]:
# write code here
%%bigquery --project $project_id
# make sure you have the header above for each SQL cell

# here's an example of a SQL query using the NCAA basketball dataset in bigquery-public-data
SELECT id, market, name, mascot, mascot_name
FROM `bigquery-public-data.ncaa_basketball.mascots`
LIMIT 5;


ERROR:
 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/$project_id/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: 72d9638d-e604-4c6c-b733-cbc80734caf6



Rank encounter classes by their decreasing total claims cost and the row count by increasing total claims cost

In [12]:
query = f"""
SELECT e.PATIENT, e.PROVIDER, e.PAYER, e.DESCRIPTION, e.TOTAL_CLAIM_COST, e.ENCOUNTERCLASS,
  RANK() OVER (
    PARTITION BY e.ENCOUNTERCLASS
    ORDER BY e.TOTAL_CLAIM_COST DESC
  ) AS Rank,
  ROW_NUMBER() OVER (
    PARTITION BY e.ENCOUNTERCLASS
    ORDER BY e.TOTAL_CLAIM_COST ASC
  ) AS RowNumber
  FROM `{PROJECT_ID}.{DATASET_ID}.encounters_ext` e
"""
df = bq_client.query(query).to_dataframe()
display(df)

,PATIENT,PROVIDER,PAYER,DESCRIPTION,TOTAL_CLAIM_COST,ENCOUNTERCLASS,Rank,RowNumber
0,4235786a-c266-ca7e-e0b1-bf9157b6104c,e6b3e4b1-6b03-3264-a5c5-7d1d86a108de,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,Telephone encounter (procedure),75.00,virtual,6537,13
1,4235786a-c266-ca7e-e0b1-bf9157b6104c,e6b3e4b1-6b03-3264-a5c5-7d1d86a108de,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,Telephone encounter (procedure),75.00,virtual,6537,25
2,55f9946d-4501-83ed-be3e-6b449fa878cb,762d8c04-7e6c-33a7-a833-fff7d5051bf2,a735bf55-83e9-331a-899d-a82a60b9f60c,Encounter for problem (procedure),75.00,virtual,6537,37
3,92b9f4d5-f424-215a-935c-e437df6e7835,89a28728-dae8-3f48-9832-55644cf2614f,d18ef2e6-ef40-324c-be54-34a5ee865625,Telephone encounter (procedure),75.00,virtual,6537,41
4,92b9f4d5-f424-215a-935c-e437df6e7835,89a28728-dae8-3f48-9832-55644cf2614f,d18ef2e6-ef40-324c-be54-34a5ee865625,Telephone encounter (procedure),75.00,virtual,6537,64
...,...,...,...,...,...,...,...,...
4850443,fc841fa5-df7c-89c3-5f66-1756dfc3af55,6b79fabb-3fd4-33a3-8a93-129c04a32200,a735bf55-83e9-331a-899d-a82a60b9f60c,Encounter for problem (procedure),1855.47,ambulatory,774555,2046472
4850444,dc252e25-1055-badc-3953-ec65dabcb565,962363d4-e85c-3f49-b912-ae38ea165b3b,df166300-5a78-3502-a46a-832842197811,Encounter for problem (procedure),1855.58,ambulatory,774551,2046476
4850445,6d756df8-6fcd-67fc-ad68-46803351ba4a,31327e86-709e-3618-9a3b-9789ed63581f,a735bf55-83e9-331a-899d-a82a60b9f60c,Encounter for problem (procedure),1855.89,ambulatory,774506,2046502
4850446,daf86709-37ca-2f76-737a-e6c4b7a5939d,adcc5bc4-be25-36e9-9a16-9e7531e2b5ff,d31fccc3-1767-390d-966a-22a5156f4219,Prenatal visit (regime/therapy),1855.96,ambulatory,774484,2046532


### Window Function 2:


Rank the insureres based on their average covered amount during different member payer tranistions

In [13]:
query = f"""
WITH payer_stats AS (
  SELECT
    p.NAME AS payer_name,
    SAFE_DIVIDE(
      p.AMOUNT_COVERED,
      p.COVERED_ENCOUNTERS
    ) AS avg_covered_amount
  FROM `{PROJECT_ID}.{DATASET_ID}.payers_ext` AS p
  WHERE p.COVERED_ENCOUNTERS > 0
)
SELECT
  payer_name,
  avg_covered_amount,
  RANK() OVER (PARTITION BY payer_name ORDER BY avg_covered_amount DESC) AS payer_rank
FROM payer_stats
ORDER BY avg_covered_amount DESC;
"""
df = bq_client.query(query).to_dataframe()
display(df)

,payer_name,avg_covered_amount,payer_rank
0,Blue Cross Blue Shield,30932.498824,1
1,Blue Cross Blue Shield,14582.763882,2
2,Blue Cross Blue Shield,10382.371045,3
3,Blue Cross Blue Shield,10135.290395,4
4,Blue Cross Blue Shield,9839.687388,5
...,...,...,...
389,Aetna,1570.795235,41
390,Aetna,1562.132459,42
391,Aetna,1545.386000,43
392,Aetna,1465.326200,44


Please copy and paste your debug table image here for this query. Otherwise delete this cell.

In [14]:
# write code here
%%bigquery --project $project_id
# make sure you have the header above for each SQL cell

# here's an example of a SQL query using the NCAA basketball dataset in bigquery-public-data
SELECT id, market, name, mascot, mascot_name
FROM `bigquery-public-data.ncaa_basketball.mascots`
LIMIT 5;


ERROR:
 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/$project_id/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: 160a69a3-dd65-4292-a1c9-73584b7380ed



### Window Function 3:


Please copy and paste your debug table image here for this query. Otherwise delete this cell.

In [15]:
# write code here
%%bigquery --project $project_id
# make sure you have the header above for each SQL cell

# here's an example of a SQL query using the NCAA basketball dataset in bigquery-public-data
SELECT id, market, name, mascot, mascot_name
FROM `bigquery-public-data.ncaa_basketball.mascots`
LIMIT 5;


ERROR:
 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/$project_id/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: b9222a5f-0097-406b-aaf7-06cc095ad3ff



Ranking the given base cost depending on the Reason Description

In [16]:
query = f"""
SELECT m.REASONDESCRIPTION, m.DESCRIPTION, m.BASE_COST,
RANK() OVER (
  PARTITION BY m.REASONDESCRIPTION
  ORDER BY m.BASE_COST DESC
) AS rank
FROM `{PROJECT_ID}.{DATASET_ID}.medications_ext` m
WHERE
  m.REASONDESCRIPTION IS NOT NULL
  AND
  m.DESCRIPTION IS NOT NULL
  AND
  m.BASE_COST > 0
ORDER BY m.BASE_COST DESC
"""
df = bq_client.query(query).to_dataframe()
display(df)

,REASONDESCRIPTION,DESCRIPTION,BASE_COST,rank
0,Alzheimer's disease (disorder),Tacrine 10 MG Oral Capsule,62289.00,1
1,Diabetes mellitus type 2 (disorder),3 ML liraglutide 6 MG/ML Pen Injector,61368.00,1
2,Diabetes mellitus type 2 (disorder),3 ML liraglutide 6 MG/ML Pen Injector,59912.76,2
3,Diabetes mellitus type 2 (disorder),3 ML liraglutide 6 MG/ML Pen Injector,59386.60,3
4,Diabetes mellitus type 2 (disorder),3 ML liraglutide 6 MG/ML Pen Injector,57631.67,4
...,...,...,...,...
5222584,Disease caused by severe acute respiratory syn...,Losartan Potassium 25 MG Oral Tablet,0.01,441
5222585,Infection caused by Pseudomonas aeruginosa (di...,20 ML Ciprofloxacin 10 MG/ML Injection,0.01,1
5222586,Infection caused by Pseudomonas aeruginosa (di...,20 ML Ciprofloxacin 10 MG/ML Injection,0.01,1
5222587,Infection caused by Pseudomonas aeruginosa (di...,20 ML Ciprofloxacin 10 MG/ML Injection,0.01,1


# Section 4: Exploring Central Questions

*[Please delete when done reading]Friendly reminder: Do not forget comments!!!*

## Question 1: [Your question]

sql query and analysis

In [17]:
query = f"""
WITH encounters_with_duration AS ( -- get duration of each encounter in minutes
  SELECT
    p.STATE,
    e.PAYER AS insurer,
    TIMESTAMP_DIFF(
      SAFE_CAST(e.STOP  AS TIMESTAMP),
      SAFE_CAST(e.START AS TIMESTAMP),
      MINUTE
    ) AS duration_min
  FROM `{PROJECT_ID}.{DATASET_ID}.encounters_ext` e
  JOIN `{PROJECT_ID}.{DATASET_ID}.patients_ext`  p
    ON p.Id = e.PATIENT
  WHERE
    SAFE_CAST(e.START AS TIMESTAMP) IS NOT NULL
    AND SAFE_CAST(e.STOP  AS TIMESTAMP) IS NOT NULL
    AND SAFE_CAST(e.STOP  AS TIMESTAMP) > SAFE_CAST(e.START AS TIMESTAMP)
),
avg_by_state_insurer AS ( -- get avg duration of each encounter by state and insurer
  SELECT
    STATE,
    insurer,
    AVG(duration_min) AS avg_duration_min,
    COUNT(*)          AS n_encounters
  FROM encounters_with_duration
  GROUP BY STATE, insurer
),
ranked_avg_by_state_insurer AS ( -- rank the duration of each encounter
  SELECT
    STATE,
    insurer,
    avg_duration_min,
    n_encounters,
    RANK() OVER (PARTITION BY STATE ORDER BY avg_duration_min DESC) as rank_num
  FROM avg_by_state_insurer
)
SELECT DISTINCT -- select out the top ranking, merging in the givne payer name
  rabsi.STATE,
  p.NAME AS payer_name,
  rabsi.avg_duration_min,
  rabsi.n_encounters,
  rabsi.rank_num
FROM ranked_avg_by_state_insurer rabsi
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.payers_ext` p
  ON rabsi.insurer = p.Id
WHERE rabsi.rank_num = 1
ORDER BY rabsi.STATE
"""
avg_by_state_insurer = bq_client.query(query).to_dataframe()
display(avg_by_state_insurer)

,STATE,payer_name,avg_duration_min,n_encounters,rank_num
0,Alabama,Medicare,542.643506,78237,1
1,Alaska,Dual Eligible,616.091471,6789,1
2,Arizona,Medicare,606.417511,53007,1
3,Arkansas,Dual Eligible,555.791520,17359,1
4,California,Dual Eligible,854.836842,8740,1
5,Colorado,Medicare,624.280149,55210,1
6,Connecticut,Dual Eligible,610.504457,10657,1
7,Delaware,Medicare,584.195908,62514,1
8,Florida,Medicare,777.397151,78630,1
9,Georgia,Medicare,569.260867,74352,1


For the 40 / 50 states that are in the dataset, this analysis tells you what payors in the healthcare system merit the greatest amount of time spent with a given patient normalized by state

This analysis is irrespective of cost, lets also bring in the msot expensive insurer by state with a similar query in order to see what might be the best insurance to actually get by weighing these two factors

In [18]:
query = f"""
WITH encounters_prices AS (
  SELECT e.PATIENT, e.TOTAL_CLAIM_COST, pay.NAME
  FROM `{PROJECT_ID}.{DATASET_ID}.encounters_ext` e
  LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.payers_ext` pay
    ON e.PAYER = pay.Id
),
encounters_price_by_state AS (
  SELECT e.PATIENT, e.TOTAL_CLAIM_COST, e.NAME, p.STATE
  FROM encounters_prices e
  LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.patients_ext` p
    ON e.PATIENT = p.Id
),
state_insurer_price AS (
  SELECT
    STATE,
    NAME,
    AVG(TOTAL_CLAIM_COST) AS avg_price
  FROM encounters_price_by_state
  WHERE NAME IS NOT NULL AND NAME != "NO_INSURANCE"
  GROUP BY STATE, NAME
),
rank_state_insurer_price AS (
  SELECT
    STATE,
    NAME,
    avg_price,
    RANK() OVER
     (
      PARTITION BY STATE ORDER BY avg_price ASC
      ) as rank_num
  FROM state_insurer_price
)
SELECT DISTINCT -- select out the top ranking, merging in the givne payer name
  rspi.STATE,
  rspi.NAME,
  rspi.avg_price,
  rspi.rank_num
FROM rank_state_insurer_price rspi
WHERE rspi.rank_num = 1
ORDER BY rspi.STATE
"""
best_price_by_insurance = bq_client.query(query).to_dataframe()
display(best_price_by_insurance)

,STATE,NAME,avg_price,rank_num
0,Alabama,Dual Eligible,1490.108212,1
1,Alaska,Aetna,3152.179638,1
2,Arizona,Dual Eligible,2685.983766,1
3,Arkansas,Blue Cross Blue Shield,2011.151838,1
4,California,Medicaid,2985.891551,1
5,Colorado,UnitedHealthcare,2327.084264,1
6,Connecticut,Cigna Health,2310.425445,1
7,Delaware,Aetna,2653.864311,1
8,Florida,Cigna Health,2776.225932,1
9,Georgia,Aetna,1930.756464,1


## Question 2: [Your question]

sql query and analysis

Moving forward to see how many interactions thata given income group has with the healhtcare system, we also need to normalize this to the number of total patients in each income group.

In [19]:
query = f"""
WITH stratified_patients AS (
  SELECT
    p.Id,
    SAFE_CAST(p.INCOME AS FLOAT64) AS income,
    CASE
      WHEN SAFE_CAST(p.INCOME AS FLOAT64) <  80000  THEN 'Low Income'
      WHEN SAFE_CAST(p.INCOME AS FLOAT64) <= 250000 THEN 'Mid Income'
      WHEN SAFE_CAST(p.INCOME AS FLOAT64) >  250000 THEN 'High Income'
      ELSE 'Unknown'
    END AS income_class
  FROM `{PROJECT_ID}.{DATASET_ID}.patients_ext` AS p
),
encounters_by_patient AS (
  -- one row per patient with their encounter count
  SELECT
    sp.income_class,
    sp.Id AS patient_id,
    sp.income,
    COUNT(e.Id) AS encounter_count
  FROM stratified_patients sp
  LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.encounters_ext` e
    ON e.PATIENT = sp.Id
  GROUP BY sp.income_class, sp.Id, sp.income
),
class_norm AS (
  -- aggregate to income class and normalize
  SELECT
    income_class,
    COUNT(*) AS patient_count,
    SUM(encounter_count) AS total_encounters,
    SAFE_DIVIDE(SUM(encounter_count), COUNT(*)) AS avg_encounters_per_patient
  FROM encounters_by_patient
  GROUP BY income_class
)
SELECT
  income_class,
  patient_count,
  total_encounters,
  avg_encounters_per_patient
FROM class_norm
ORDER BY
  CASE income_class
    WHEN 'Low Income' THEN 1
    WHEN 'Mid Income' THEN 2
    WHEN 'High Income' THEN 3
    ELSE 4
  END;

"""
df = bq_client.query(query).to_dataframe()
display(df)

,income_class,patient_count,total_encounters,avg_encounters_per_patient
0,Low Income,42051,3401862,80.898480
1,Mid Income,16609,1245400,74.983443
2,High Income,2548,203186,79.743328


In [20]:
query = f"""
WITH stratified_patients AS (
  SELECT
    p.Id,
    SAFE_CAST(p.INCOME AS FLOAT64) AS income,
    CASE
      WHEN SAFE_CAST(p.INCOME AS FLOAT64) <  80000  THEN 'Low Income'
      WHEN SAFE_CAST(p.INCOME AS FLOAT64) <= 250000 THEN 'Mid Income'
      WHEN SAFE_CAST(p.INCOME AS FLOAT64) >  250000 THEN 'High Income'
      ELSE 'Unknown'
    END AS income_class
  FROM `{PROJECT_ID}.{DATASET_ID}.patients_ext` AS p
),
price_by_patients AS (
  -- one row per patient with their encounter count
  SELECT
    sp.income_class,
    sp.Id AS patient_id,
    sp.income,
    SUM(e.BASE_ENCOUNTER_COST) AS sum_base_encounter_cost
  FROM stratified_patients sp
  LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.encounters_ext` e
    ON e.PATIENT = sp.Id
  GROUP BY sp.income_class, sp.Id, sp.income
),
class_income_normed AS (
  -- aggregate to income class and normalize
  SELECT
    income_class,
    COUNT(*) AS patient_count,
    SAFE_DIVIDE(SUM(sum_base_encounter_cost), COUNT(*)) AS avg_base_price_per_patient
  FROM price_by_patients
  GROUP BY income_class
)
SELECT
  income_class,
  patient_count,
  avg_base_price_per_patient
FROM class_income_normed
ORDER BY
  CASE income_class
    WHEN 'Low Income' THEN 1
    WHEN 'Mid Income' THEN 2
    WHEN 'High Income' THEN 3
    ELSE 4
  END;

"""
df = bq_client.query(query).to_dataframe()
display(df)

,income_class,patient_count,avg_base_price_per_patient
0,Low Income,42051,7478.717056
1,Mid Income,16609,7057.538988
2,High Income,2548,7590.136464


## Question 3: [Your question]

sql query and analysis

In [21]:
query = f"""-- 30-day revisit rate by (STATE, insurer)
WITH enc AS (
  SELECT
    e.Id,
    e.PATIENT,
    e.START AS start_ts,
    e.PAYER
  FROM `{PROJECT_ID}.{DATASET_ID}.encounters_ext` e
  WHERE e.START IS NOT NULL
),
enc_with_next AS (
  SELECT
    Id,
    PATIENT,
    PAYER,
    start_ts,
    LEAD(start_ts) OVER (
      PARTITION BY PATIENT ORDER BY start_ts
      ) AS next_start_ts
  FROM enc
),
flag_30d AS (
  SELECT
    x.PATIENT,
    x.PAYER,
    -- index encounter (the one we look forward from)
    x.start_ts AS index_start,
    -- revisit within 30 days?
    IF(
      x.next_start_ts IS NOT NULL
      AND TIMESTAMP_DIFF(x.next_start_ts, x.start_ts, DAY) <= 30,
      1, 0
    ) AS revisit_30d
  FROM enc_with_next x
),
with_state_payer AS (
  SELECT
    p.STATE,
    # COALESCE(pay.NAME, 'UNKNOWN_PAYER') AS insurer,
    pay.NAME AS insurer,
    f.revisit_30d
  FROM flag_30d f
  JOIN `{PROJECT_ID}.{DATASET_ID}.patients_ext` p
    ON p.Id = f.PATIENT
  LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.payers_ext` pay
    ON pay.Id = f.PAYER
  WHERE pay.NAME IS NOT NULL AND NAME != "NO_INSURANCE"

)
SELECT
  STATE,
  insurer,
  COUNT(*) AS index_encounters,
  SUM(revisit_30d) AS revisits_within_30d,
  SAFE_DIVIDE(SUM(revisit_30d), COUNT(*)) AS revisit_rate_30d
FROM with_state_payer
GROUP BY STATE, insurer
# HAVING COUNT(*) >= 10                -- optional: suppress tiny cells
ORDER BY revisit_rate_30d DESC, index_encounters DESC;
"""

df = bq_client.query(query).to_dataframe()
display(df)

,STATE,insurer,index_encounters,revisits_within_30d,revisit_rate_30d
0,Alabama,Dual Eligible,56364,51040,0.905543
1,Florida,Dual Eligible,24816,21560,0.868794
2,Kansas,Dual Eligible,189772,163724,0.862741
3,Idaho,Dual Eligible,712272,608652,0.854522
4,Wyoming,Medicare,55880,47344,0.847244
...,...,...,...,...,...
362,Wyoming,Anthem,1408,484,0.343750
363,Wyoming,Cigna Health,572,176,0.307692
364,Wyoming,UnitedHealthcare,176,44,0.250000
365,Wyoming,Blue Cross Blue Shield,748,132,0.176471


# Section 5: Takeaways

---

*TODO: Final conclusions based on the rest of your project*

---